**Goal:** The aim of the challenge was to provide an opportunity for the development, testing and evaluation of AI models for automatic classification of abnormalities captured in video capsule endoscopy (VCE) video frames. It promotes the development of vendor-independent and generalized AI-based models for automatic abnormality classification pipeline with 10 class labels:
1.	Angioectasia
2.	Bleeding
3.	Erosion
4.	Erythema
5.	Foreign body
6.	Lymphangiectasia
7.	Polyp
8.	Ulcer
9.	Worms
10.	Normal

**Evaluation**
1.	Goal Metric
    •	Balanced Accuracy
    •	Mean AUC
2.	Other Metrics
    •	AUC-ROC
    •	Specificity
    •	Mean Specificity
    •	F1 Score
    •	Mean F1 Score
    •	Average Precision
    •	Mean Average Precision


**Importing necessary libraries**

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os

**Define dataset directories**

In [20]:
train_dir = r'C:\Users\Mokshda Sharma\Desktop\My Projects\Capsule vision\training'
val_dir = r'C:\Users\Mokshda Sharma\Desktop\My Projects\Capsule vision\validation'
test_dir = r'C:\Users\Mokshda Sharma\Desktop\My Projects\Capsule vision\testing'

**Loading images and preprocessing them (basically resizing)**

In [21]:
# Image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [22]:
# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=False)

Found 37607 images belonging to 10 classes.
Found 16132 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


**Define CNN Model**

With the help of transfer learning, using a base model of MobileNetV2 and training a basic CNN model with that base model for better results.

In [24]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze pre-trained layers

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


**Compile the model**

In [25]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Train and save the model**

In [ ]:
# Train the model
model.fit(train_generator, 
          validation_data=val_generator, 
          epochs=10, 
          verbose=1)


c:\Users\Mokshda Sharma\Desktop\My Projects\Capsule vision\env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 1190s 1s/step - accuracy: 0.7736 - loss: 0.7857 - val_accuracy: 0.8026 - val_loss: 0.6385
Epoch 2/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 660s 561ms/step - accuracy: 0.8198 - loss: 0.5711 - val_accuracy: 0.8209 - val_loss: 0.5679
Epoch 3/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 1515s 1s/step - accuracy: 0.8291 - loss: 0.5301 - val_accuracy: 0.8121 - val_loss: 0.6024
Epoch 4/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 654s 556ms/step - accuracy: 0.8363 - loss: 0.5055 - val_accuracy: 0.8281 - val_loss: 0.5254
Epoch 5/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 1025s 872ms/step - accuracy: 0.8392 - loss: 0.4849 - val_accuracy: 0.8288 - val_loss: 0.5387
Epoch 6/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 1141s 969ms/step - accuracy: 0.8442 - loss: 0.4637 - val_accuracy: 0.8339 - val_loss: 0.5245
Epoch 7/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 656s 558ms/step - accuracy: 0.8515 - loss: 0.4478 - val_accuracy: 0.8260 - val_loss: 0.5505
Epoch 8/10
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 652s 555ms/step - accu

**Model Evaluation**

In [28]:
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc:.4f}")
print(f"Validation Loss: {loss:.4f}")

505/505 ━━━━━━━━━━━━━━━━━━━━ 125s 247ms/step - accuracy: 0.8405 - loss: 0.4869
Validation Accuracy: 0.8399
Validation Loss: 0.4874


**Using model on test data**

In [29]:
# Function to load and preprocess test images
def load_test_images(test_dir):
    test_images = []
    image_filenames = []
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = load_img(img_path, target_size=(224, 224))  # Ensure correct size
        img_array = img_to_array(img) / 255.0  # Normalize
        test_images.append(img_array)
        image_filenames.append(img_name)
    return np.array(test_images), image_filenames

# Load test images
test_dir = r'C:\Users\Mokshda Sharma\Desktop\My Projects\Capsule vision\testing'
test_images, test_filenames = load_test_images(test_dir)

# Make predictions
predictions = model.predict(test_images)

# Get class labels
class_labels = list(train_generator.class_indices.keys())

# Print predictions
for i, pred in enumerate(predictions):
    predicted_class = class_labels[np.argmax(pred)]
    print(f"Image: {test_filenames[i]} -> Predicted: {predicted_class}")


138/138 ━━━━━━━━━━━━━━━━━━━━ 35s 235ms/step
Image: 00Z0Xo99wp.jpg -> Predicted: Normal
Image: 02hvtCoV9C.jpg -> Predicted: Normal
Image: 03pjR51twC.jpg -> Predicted: Normal
Image: 03UqLvuk8v.jpg -> Predicted: Polyp
Image: 04jiugLcCI.jpg -> Predicted: Normal
Image: 04tn1G74xo.jpg -> Predicted: Normal
Image: 05c6TBcXSu.jpg -> Predicted: Normal
Image: 06fIH8amoG.jpg -> Predicted: Normal
Image: 06XFjXueSX.jpg -> Predicted: Erosion
Image: 07jOML1ogZ.jpg -> Predicted: Normal
Image: 07nd6BLrq1.jpg -> Predicted: Normal
Image: 07rfIWuhqW.jpg -> Predicted: Polyp
Image: 08EWI6eGVX.jpg -> Predicted: Normal
Image: 09LTohVSRn.jpg -> Predicted: Normal
Image: 0a0lZMebPu.jpg -> Predicted: Normal
Image: 0A1nx29BXs.jpg -> Predicted: Normal
Image: 0cZrtERnYM.jpg -> Predicted: Normal
Image: 0dIQ3E45xb.jpg -> Predicted: Normal
Image: 0DQ7QCq2cM.jpg -> Predicted: Normal
Image: 0dYnoY95dR.jpg -> Predicted: Normal
Image: 0e97xILFmD.jpg -> Predicted: Normal
Image: 0etZQ1V2B5.jpg -> Predicted: Normal
Image: 0FNJ

**Saves the model**

In [30]:
model.save("disease_model.h5")  # Saves the entire model (architecture + weights)